# Building a Text-to-SQL ReAct Agent

We'll build a **ReAct agent** using [DSPy](https://dspy.ai/) that translates natural language questions into SQL queries against a quick-commerce DuckDB database (order data from Pune, India).

**What we'll cover:**
1. **Build** — Connect to DuckDB, define a DSPy ReAct agent
2. **Evaluate** — Score the agent using metrics
4. **Optimize** — Use DSPy optimizers to automatically tune the agent

In [ ]:
# Imports
import os
import sys

os.environ["TQDM_DISABLE"] = "1"
sys.path.insert(0, "..")

from typing import Literal

import dspy
import duckdb
from compare import display_comparison
from dotenv import load_dotenv
from eval_utils import (
    get_last_analytical_sql,
    run_eval,
    run_single_eval,
    save_eval,
)
from trajectory import save_trajectory

from data.eval_dataset import DATASET

load_dotenv()

True

In [3]:
# DuckDB: connect to the database file
db_conn = duckdb.connect("../data/qc_pune.duckdb", read_only=True)


def execute_sql(sql: str, limit: int = 50) -> str:
    """Execute a SQL query against the DuckDB database.

    Use SHOW TABLES to discover tables, DESCRIBE table_name for schema.
    Use LIMIT on exploratory queries. Write valid DuckDB SQL.
    """
    try:
        result = db_conn.execute(sql).fetchall()
        if not result:
            return "(No rows returned)"

        columns = [desc[0] for desc in db_conn.description]
        header = " | ".join(columns)
        rows = "\n".join(" | ".join(str(v) for v in row) for row in result[:limit])
        total = len(result)
        note = f"\n... ({total} total rows)" if total > limit else ""
        return f"{header}\n{rows}{note}"

    except Exception as e:
        return f"SQL Error: {e}"

In [37]:
sql_query = "SHOW TABLES;"
print(execute_sql(sql_query))

name
consumers
itemized_orders
product_catalogue
stores


```text
┌──────────────────────────────┐                ┌──────────────────────────────────────┐
│          consumers           │                │           itemized_orders            │
├──────────────────────────────┤                ├──────────────────────────────────────┤
│ VARCHAR  consumer_id     PK  │◀┐              │ VARCHAR   order_id              PK   │
│ VARCHAR  address_id      PK2 │◀┼┐             │ INTEGER   order_line_id         PK2  │
│ VARCHAR  gender              │ └│─────────────│ VARCHAR   consumer_id           FK   │
│ INTEGER  age                 │  └─────────────│ VARCHAR   address_id            FK   │
│ DOUBLE   latitude            │            ┌───│ VARCHAR   store_id              FK   │
│ DOUBLE   longitude           │          ┌─│───│ VARCHAR   product_id            FK   │
│ VARCHAR  address             │          │ │   │ INTEGER   quantity                   │
│ VARCHAR  pincode             │          │ │   │ DOUBLE    mrp                        │
│ VARCHAR  nearest_store_id FK │──┐       │ │   │ DOUBLE    discount                   │
└──────────────────────────────┘  │       │ │   │ DOUBLE    item_total                 │
                                  │       │ │   │ TIMESTAMP order_timestamp            │
┌──────────────────────────────┐  │       │ │   │ VARCHAR   order_status               │
│      product_catalogue       │  │       │ │   │ VARCHAR   cancel_reason              │
├──────────────────────────────┤  │       │ │   │ INTEGER   committed_delivery_secs    │
│ VARCHAR  product_id      PK  │◀─┼───────┘ │   │ INTEGER   actual_delivery_secs       │
│ VARCHAR  product_name        │  │         │   │ DOUBLE    distance_km                │
│ VARCHAR  brand_name          │  │         │   └──────────────────────────────────────┘
│ VARCHAR  brand_id            │  │         │
│ VARCHAR  category            │  │         │   ┌──────────────────────────────────────┐
│ VARCHAR  sub_category        │  │         │   │                stores                │
│ VARCHAR  size                │  │         │   ├──────────────────────────────────────┤
│ DOUBLE   mrp                 │  └─────────└──▶│ VARCHAR  store_id              PK    │
│ VARCHAR  category_code       │                │ VARCHAR  store_name                  │
│ VARCHAR  sub_category_code   │                │ DOUBLE   latitude                    │
└──────────────────────────────┘                │ DOUBLE   longitude                   │
                                                │ DOUBLE   radius_km                   │
                                                │ VARCHAR  pincode                     │
                                                │ VARCHAR  area                        │
                                                │ VARCHAR  serviceable_area            │
                                                └──────────────────────────────────────┘
```

In [ ]:
# Try it yourself
sql_query = """"""
print(execute_sql(sql_query))

In [ ]:
# DSPy LM via OpenRouter
lm = dspy.LM(
    "openrouter/google/gemini-2.5-flash",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    max_tokens=100000,
    cache=False,
)
dspy.configure(lm=lm, adapter=dspy.ChatAdapter(), track_usage=True)

## Text-to-SQL ReAct Agent

In [5]:
# ReAct Agent for Data Analysis
class AnalystSignature(dspy.Signature):
    """You are a data analyst working with a DuckDB database.

    Given a user's question, explore the database schema, write and execute
    SQL queries, and provide a clear, accurate answer grounded in the data.

    Start by discovering available tables (SHOW TABLES) and their schemas
    (DESCRIBE table_name) before writing analytical queries.
    """

    question: str = dspy.InputField(desc="A natural language question about the data")
    answer: str = dspy.OutputField(desc="A concise, data-backed answer to the question")


agent_v1 = dspy.ReAct(AnalystSignature, tools=[execute_sql], max_iters=10)

In [11]:
weekend_orders = agent_v1(
    question="Which top 3 product categories that are often ordered on weekends? Give total order count for each category as well."
)
print(weekend_orders.answer)

The top 3 product categories that are most often ordered on weekends are:
1. **Personal Care**: 193,562 orders
2. **Instant & Ready to Eat**: 192,188 orders
3. **Beverages**: 192,144 orders


In [16]:
# Save the trajectory and LM usage to a markdown file
md_path = save_trajectory(
    weekend_orders.trajectory,
    weekend_orders.get_lm_usage(),
    name="weekend_orders_agent_v1",
)

## Marker Agent

Once the agent has answered, what this agent does is:
- Identify what the SQL query actually does — a simple summary so you can verify it understood your question
- Does it actually answer what you asked? — sometimes the agent fetches data that looks related but doesn't quite answer the question
- Checks what might be missing — e.g. it filtered out cancelled orders when you wanted all orders, or it counted line items instead of unique orders
- How confident we are — a gut-check on whether you should trust the answer or double-check it yourself

We need this because the agent can return confident-sounding answers that are subtly wrong — wrong grouping, missing filters, counting rows instead of orders. Without an check, those errors silently reach the user.

In [6]:
class MarkerSignature(dspy.Signature):
    """Analyze the agent's SQL query and answer for a given user question.

    Explain what the SQL does in plain English, assess whether the output
    answers the user's question, identify any gaps, and rate your confidence.
    """

    user_query: str = dspy.InputField(desc="The original natural language question")
    final_generated_sql: str = dspy.InputField(
        desc="The final analytical SQL the agent executed"
    )
    answer: str = dspy.InputField(desc="The agent's textual answer to the question")
    humane_version_of_sql: str = dspy.OutputField(
        desc="Plain-English explanation of what the SQL does without mentioning techincal details so that it can be understood by non-technical person. Give list in markdown format if there are multiple steps."
    )
    does_op_answer_user_query: str = dspy.OutputField(
        desc="Yes/No with brief justification — does the output answer the question?"
    )
    gaps_in_answer: str = dspy.OutputField(
        desc="What is missing, wrong, or could be improved in the answer"
    )
    confidence: Literal["low", "medium", "high"] = dspy.OutputField(
        desc="Confidence in the answer quality"
    )


marker = dspy.ChainOfThought(MarkerSignature)

In [13]:
# Run marker on the weekend_orders result
agent_sql = get_last_analytical_sql(weekend_orders.trajectory)

marked = marker(
    user_query="Which top 3 product categories that are often ordered on weekends?",
    final_generated_sql=agent_sql or "(no SQL)",
    answer=weekend_orders.answer,
)

print(f"SQL Explanation:\n{marked.humane_version_of_sql}\n")
print(f"Answers the question?:\n{marked.does_op_answer_user_query}\n")
print(f"Gaps:\n{marked.gaps_in_answer}\n")
print(f"Confidence:\n{marked.confidence}")

SQL Explanation:
To find the top 3 product categories ordered on weekends, the system did the following:
1. Looked at all customer orders.
2. Filtered these orders to only include those placed on Saturdays or Sundays.
3. For each of these weekend orders, identified the category of the product purchased.
4. Counted how many unique orders each product category received on weekends.
5. Ranked the product categories from most to least ordered on weekends.
6. Picked out the top 3 categories from this ranked list.

Answers the question?:
Yes, the output directly identifies the top 3 product categories most often ordered on weekends, as requested by the user.

Gaps:
There are no significant gaps in the answer. It directly addresses the user's query clearly and concisely.

Confidence:
high


In [ ]:
# Try it yourself
query_result = agent_v1(question="")
print(query_result.answer)

## Evals

### What are Evals?



### Metric 1: Tool Efficiency

**Why we need this?**
- Every tool call is like an LLM call — it costs money and adds latency. An agent that takes 8 calls to answer "how many products are there?" works, but it's 4x slower and 4x more expensive than one that does it in 2 tool calls.
- This metric tells you whether the agent is exploring the database efficiently or burning tokens on unnecessary SHOW TABLES / DESCRIBE / trial-and-error queries before getting to the actual answer.


In [7]:
def tool_efficiency(trajectory: dict) -> float:
    """Score based on number of tool calls. Fewer is better."""
    i = 0
    while f"observation_{i}" in trajectory:
        i += 1
    if i <= 4:
        return 1.0
    if i <= 7:
        return 0.5
    return 0.25

In [18]:
print(f"Tool efficiency: {tool_efficiency(weekend_orders.trajectory):.2f}")

Tool efficiency: 0.50


### Metric 2: SQL Validity

**Why we need this?**

Measures what fraction of the agent's SQL queries actually ran without errors. If the agent writes SELECT * FROM ordes (typo), that's a failed call — wasted money, added latency, and it means the agent is guessing at table/column names instead of knowing them.

In [8]:
def sql_validity(trajectory: dict) -> float:
    """Fraction of SQL tool calls that did NOT return an error."""
    errors = total = 0
    i = 0
    while f"observation_{i}" in trajectory:
        if trajectory.get(f"tool_name_{i}") == "execute_sql":
            total += 1
            if trajectory[f"observation_{i}"].startswith("SQL Error:"):
                errors += 1
        i += 1
    return (total - errors) / total if total else 1.0

In [20]:
print(f"SQL validity: {sql_validity(weekend_orders.trajectory):.2f}")

SQL validity: 1.00


### Metric 3: Error Recovery

**Why we need this?**

An agent that makes a mistake isn't necessarily bad, as long as it fixes itself. This measures whether the agent recovered after hitting an error. A low score here means the agent is fragile — one wrong query and it falls apart.


In [9]:
def error_recovery(trajectory: dict, answer: str) -> float:
    """Did the agent self-correct after SQL errors?"""
    errors = 0
    i = 0
    while f"observation_{i}" in trajectory:
        if trajectory.get(f"tool_name_{i}") == "execute_sql":
            if trajectory[f"observation_{i}"].startswith("SQL Error:"):
                errors += 1
        i += 1
    if errors == 0:
        return 1.0
    if answer and answer != "(No response)":
        return 0.75
    return 0.0

In [22]:
print(
    f"Error recovery: {error_recovery(weekend_orders.trajectory, weekend_orders.answer):.2f}"
)

Error recovery: 1.00


### Metric 4: Answer Quality

**Why we need this?**

- The other three metrics tell you how the agent worked — was it fast, clean, resilient. But none of them tell you if the answer is actually correct.
- An agent can score 1.0 on efficiency, 1.0 on SQL validity, 1.0 on recovery — and still return completely wrong data. It wrote valid SQL, it did it in 2 calls, it never errored... but it counted line items instead of orders, so the number is 3x too high.
- Answer Quality is the only metric that catches that. It's the difference between "the agent ran smoothly" and "the agent gave you correct output."

In [10]:
class AnswerQualitySignature(dspy.Signature):
    """Rate how well the agent answered a data analysis question.

    Evaluate both the SQL approach and the output data holistically.

    For the SQL query, ignore superficial differences such as:
    - Aliases and formatting
    - JOIN syntax variations
    - Column order
    - Equivalent date functions
    - Additional ORDER BY/LIMIT clauses

    For the output data, ignore superficial differences such as:
    - Different column names for the same data (e.g. "store" vs "store_name")
    - Different row ordering
    - Minor rounding differences (e.g. 14.7 vs 14.71)
    - Extra columns that don't change the core answer
    - Different date/time formatting (e.g. "2025-11" vs "2025-11-01")

    Consider the answer incorrect only if the core logic or data differs:
    - Wrong GROUP BY, aggregations, filters, or JOINs
    - Significantly different row counts or wrong values
    - Missing key columns that the question asked for
    """

    question: str = dspy.InputField(desc="The natural language question")
    reference_sql: str = dspy.InputField(desc="The ground truth SQL query")
    agent_sql: str = dspy.InputField(desc="The agent-generated SQL query")
    expected_csv: str = dspy.InputField(desc="The expected output data (CSV)")
    agent_csv: str = dspy.InputField(desc="The agent-generated output data (CSV)")
    score: float = dspy.OutputField(desc="Score between 0 and 1")


judge = dspy.ChainOfThought(AnswerQualitySignature)


def answer_quality(agent_sql, reference_sql, expected_csv, question, conn):
    """Execute agent SQL and use an LLM judge to score the result."""
    if not agent_sql:
        return {"score": 0.0, "reasoning": "No analytical SQL found", "agent_csv": None}

    try:
        result_df = conn.execute(agent_sql).fetchdf()
        agent_csv_str = result_df.to_csv(index=False)
    except Exception as e:
        return {
            "score": 0.0,
            "reasoning": f"SQL execution error: {e}",
            "agent_csv": None,
        }

    agent_csv_full = agent_csv_str

    # Truncate for prompt size
    agent_lines = agent_csv_str.strip().split("\n")
    if len(agent_lines) > 51:
        agent_csv_str = (
            "\n".join(agent_lines[:51]) + f"\n... ({len(agent_lines) - 1} total rows)"
        )

    expected_lines = expected_csv.strip().split("\n")
    if len(expected_lines) > 51:
        expected_csv = (
            "\n".join(expected_lines[:51])
            + f"\n... ({len(expected_lines) - 1} total rows)"
        )

    try:
        result = judge(
            question=question,
            reference_sql=reference_sql,
            agent_sql=agent_sql,
            expected_csv=expected_csv,
            agent_csv=agent_csv_str,
        )
        return {
            "score": float(result.score),
            "reasoning": result.reasoning,
            "agent_csv": agent_csv_full,
        }
    except Exception as e:
        return {
            "score": 0.0,
            "reasoning": f"Judge error: {e}",
            "agent_csv": agent_csv_full,
        }

In [24]:
# Example: score the agent's SQL + output against reference
agent_sql = get_last_analytical_sql(weekend_orders.trajectory)
reference_sql = """
SELECT pc.category, COUNT(DISTINCT io.order_id) AS weekend_order_count
FROM itemized_orders io
JOIN product_catalogue pc ON io.product_id = pc.product_id
WHERE DAYOFWEEK(io.order_timestamp) IN (1, 7)
  AND io.order_status = 'delivered'
GROUP BY pc.category
ORDER BY weekend_order_count DESC
LIMIT 3;
"""
expected_csv = "category,weekend_order_count\nPersonal Care,76915\nInstant & Ready to Eat,75652\nBeverages,75555"

result = answer_quality(
    agent_sql=agent_sql,
    reference_sql=reference_sql,
    expected_csv=expected_csv,
    question="Which top 3 product categories are often ordered on weekends?",
    conn=db_conn,
)
print(f"Score: {result['score']}")
print(f"Reasoning: {result['reasoning']}")

Score: 0.2
Reasoning: The agent failed to include the `order_status = 'delivered'` filter. As a result, the `total_orders` counts are significantly different from the `weekend_order_count` in the expected output. While the top 3 categories are the same, the values are incorrect due to the missing filter, leading to a low score.


In [30]:
eval_results = run_eval(agent_v1, DATASET, db_conn, agent_name="agent_v1")

Running eval on 16 examples with agent_v1

#   ID                                  Difficulty   Efficiency   SQLValid   Recovery   Answer Quality
----------------------------------------------------------------------------------------------------
1   1_product_count                     easy         1.00         1.00       1.00       1.00       (28.2s)
2   2_unique_categories                 easy         1.00         1.00       1.00       1.00       (17.9s)
3   3_store_count                       easy         1.00         1.00       1.00       1.00       (5.4s)
4   4_avg_age_by_gender                 easy         1.00         1.00       1.00       0.90       (8.1s)
5   5_orders_by_status                  easy         0.50         1.00       1.00       1.00       (18.7s)
6   6_top_store_delivered               easy         0.50         1.00       1.00       0.00       (16.6s)
7   7_store_performance_scorecard       medium       0.50         1.00       1.00       0.70       (40.3s)
8   8_

## Add Your Own Eval

Use `save_eval` to create a new eval example from a question + SQL query. It runs the SQL, saves the CSV and SQL files, and returns a dataset entry you can evaluate.

In [26]:
# Example: save a new eval
new_example = save_eval(
    eval_id="17_top_brands_snacks",
    question="What are the top 3 brands by revenue in the Snacks & Munchies category?",
    sql="""
SELECT pc.brand_name, SUM(io.item_total) AS total_revenue
FROM itemized_orders io
JOIN product_catalogue pc ON io.product_id = pc.product_id
WHERE pc.category = 'Snacks & Munchies'
  AND io.order_status = 'delivered'
GROUP BY pc.brand_name
ORDER BY total_revenue DESC
LIMIT 3;
""",
    conn=db_conn,
    difficulty="medium",
    expected_tables=["itemized_orders", "product_catalogue"],
)

Saved 17_top_brands_snacks:
  SQL → eval_answer_sqls/17_top_brands_snacks.sql
  CSV → eval_answer_csvs/17_top_brands_snacks.csv (3 rows)


In [ ]:
# Try it yourself
sql_query = """"""
print(execute_sql(sql_query))

In [ ]:
# Try it yourself
new_example = save_eval(
    eval_id="",  # eg "17" or "top_brands_snack" or "17_top_brands_snacks"
    question="",  # eg "What are the top 3 brands by revenue in the Snacks & Munchies category?"
    sql="""""",  # eg the SQL query the agent generated for that question
    conn=db_conn,
    difficulty="",  # easy, medium or hard
    expected_tables=[],  # list of tables that should be used in the SQL answer, e.g. ["itemized_orders", "product_catalogue"]
)

In [27]:
# Run the agent on your new example and display comparison
r = run_single_eval(agent_v1, new_example, db_conn)

display_comparison(
    question=new_example["question"],
    reference_sql=new_example["reference_sql"],
    agent_sql=r["agent_sql"],
    expected_csv=new_example["expected_answer"],
    agent_csv=r.get("agent_csv"),
    answer_quality_score=r["answer_quality_score"],
    answer_quality_reasoning=r["answer_quality_reasoning"],
)

brand_name,total_revenue
Cadbury,12344944.56
Ferrero,10289819.37
ITC,8770143.97
brand_name,total_revenue
Cadbury,12591769.95
Ferrero,10465097.05
ITC,8943593.25


"""
I need help debugging a DSPy agent that's failing evals. I'm providing:
1. **Eval Results** — output/scores from my eval run
2. **Eval Definition** — metrics, dataset, assertions
3. **Agent Definition** — module/signature/program

## What to do:

**Understand:** Understand what the agent does and what the eval measures.

**Analyze Failures:** Identify failing cases, categorize failure modes (wrong format, hallucination, bad reasoning, tool misuse, etc.), and find patterns.

**Trace Root Causes:** For each failure pattern, trace to root cause — underspecified signature? wrong module type? bad tool descriptions? buggy eval metric?

**Suggest Fixes (Ranked):** Propose concrete fixes ranked by likely impact — exact prompt/signature changes, architecture changes, tool description improvements, eval fixes.

## Output constraints:
- No filler or preamble. Be direct.
- Use tables for failure analysis (case → failure mode → root cause → fix).
- Keep each section to 2 lines max.
- Show exact code diffs for proposed fixes, not descriptions of what to change.
- Total response should fit in 10-15 lines.

---

### Eval Results:
<paste here>

### Eval Definition:
<paste here>

### Agent Definition:
<paste here>
"""

User query, agent, tracjectory

## Improving the agent 

**Why we need it?**

Agent V1 has no idea what's in the database. Every time you ask it a question, it has to spend 2-3 calls just running SHOW TABLES and DESCRIBE to figure out what tables exist and what columns they have — before it even starts writing the actual query. That causes two problems:
- It's slow and expensive — half the tool calls are just schema discovery, not actual analysis. That's why V1's efficiency avg is 0.84.
- It still gets things wrong — reading a DESCRIBE output doesn't tell the agent that itemized_orders has one row per line item not per order, or that SLA breach means actual_delivery_secs > committed_delivery_secs. Without that context, it guesses wrong on the hard queries.

Agent V2 fixes this by passing the full schema + documentation upfront as a db_context input field. The agent skips discovery entirely and goes straight to writing the analytical query.

In [11]:
# Add the database context to the agent's instructions to further improve its performance and reduce iterations
DB_CONTEXT = """
## Database: qc_pune.duckdb (DuckDB)
Quick-commerce order data for Pune, India (~Nov 2025 – Feb 2026).

### Tables

1. consumers — Customer profiles with delivery addresses
   - consumer_id  (VARCHAR, PK) – unique customer ID, e.g. "CON-070619"
   - address_id   (VARCHAR, PK) – unique address per customer, e.g. "CON-070619-A2"
   - gender       (VARCHAR)     – customer gender
   - age          (INTEGER)     – customer age
   - latitude     (DOUBLE)      – delivery address latitude
   - longitude    (DOUBLE)      – delivery address longitude
   - address      (VARCHAR)     – full delivery address text
   - pincode      (VARCHAR)     – postal pin code
   - nearest_store_id (VARCHAR) – ID of the closest store, e.g. "STR-12"

2. itemized_orders — One row per line item in every order
   - order_id      (VARCHAR, PK)   – order identifier, e.g. "ORD-20251101-000001"
   - order_line_id (INTEGER, PK)   – line number within the order (1, 2, …)
   - consumer_id   (VARCHAR)       – FK → consumers.consumer_id
   - address_id    (VARCHAR)       – FK → consumers.address_id
   - store_id      (VARCHAR)       – FK → stores.store_id
   - product_id    (VARCHAR)       – FK → product_catalogue.product_id
   - quantity      (INTEGER)       – units ordered
   - mrp           (DOUBLE)        – maximum retail price per unit
   - discount      (DOUBLE)        – fractional discount (0.08 = 8 %)
   - item_total    (DOUBLE)        – line total after discount
   - order_timestamp (TIMESTAMP)   – when the order was placed
   - order_status  (VARCHAR)       – e.g. "delivered", "cancelled"
   - cancel_reason (VARCHAR, NULL) – reason if cancelled, else NULL
   - committed_delivery_secs (INTEGER) – promised delivery time in seconds
   - actual_delivery_secs   (INTEGER, NULL) – actual delivery time in seconds
   - distance_km   (DOUBLE)       – distance from store to customer (km)

3. product_catalogue — Master product list
   - product_id        (VARCHAR, PK) – e.g. "PRD-DB-MLK-10199"
   - product_name      (VARCHAR)     – e.g. "Britannia Milk Bread"
   - brand_name        (VARCHAR)     – e.g. "Britannia", "Dabur"
   - brand_id          (VARCHAR)     – e.g. "BRD-79948b7a"
   - category          (VARCHAR)     – top-level category, e.g. "Dairy & Breakfast", "Beverages"
   - sub_category      (VARCHAR)     – e.g. "Baby Food", "Milk", "Juices"
   - size              (VARCHAR)     – pack size, e.g. "250g", "1kg", "500ml"
   - mrp               (DOUBLE)      – listed MRP
   - category_code     (VARCHAR)     – short code, e.g. "BAB", "DB", "BV"
   - sub_category_code (VARCHAR)     – short code, e.g. "BF", "MLK", "JU"

4. stores — Store locations and service areas
   - store_id         (VARCHAR, PK) – e.g. "STR-12"
   - store_name       (VARCHAR)     – human-readable store name
   - latitude         (DOUBLE)      – store latitude
   - longitude        (DOUBLE)      – store longitude
   - radius_km        (DOUBLE)      – delivery radius in km
   - pincode          (VARCHAR)     – store pin code
   - area             (VARCHAR)     – area / neighbourhood name
   - serviceable_area (VARCHAR)     – broader serviceable zone
"""


class AnalystSignature(dspy.Signature):
    """You are a data analyst working with a DuckDB database.

    Given a user's question, write and execute SQL queries and provide a
    clear, accurate answer grounded in the data.

    Do NOT waste iterations on SHOW TABLES or DESCRIBE — go straight to analytical queries.
    """

    question: str = dspy.InputField(desc="A natural language question about the data")
    db_context: str = dspy.InputField(desc="Database schema and documentation")
    answer: str = dspy.OutputField(desc="A concise, data-backed answer to the question")


agent_v2 = dspy.ReAct(AnalystSignature, tools=[execute_sql], max_iters=10)

In [29]:
# Run the same question with the new agent that has DB context in its instructions
weekend_orders = agent_v2(
    question="Which top 3 product categories that are often ordered on weekends?",
    db_context=DB_CONTEXT,
)
print(weekend_orders.answer)

trajectory = weekend_orders.trajectory
observations = [k for k in trajectory if k.startswith("observation_")]
print(f"\nTotal iterations (with DB context): {len(observations)}")

The top 3 product categories ordered on weekends are:
1. Personal Care (189634 orders)
2. Beverages (188363 orders)
3. Instant & Ready to Eat (188342 orders)

Total iterations (with DB context): 2


In [31]:
agent_v2 = dspy.ReAct(AnalystSignature, tools=[execute_sql], max_iters=10)
eval_results = run_eval(
    agent_v2, DATASET, db_conn, agent_name="agent_v2", db_context=DB_CONTEXT
)

Running eval on 16 examples with agent_v2

#   ID                                  Difficulty   Efficiency   SQLValid   Recovery   Answer Quality
----------------------------------------------------------------------------------------------------
1   1_product_count                     easy         1.00         1.00       1.00       1.00       (6.4s)
2   2_unique_categories                 easy         1.00         1.00       1.00       1.00       (8.8s)
3   3_store_count                       easy         1.00         1.00       1.00       1.00       (4.1s)
4   4_avg_age_by_gender                 easy         1.00         1.00       1.00       0.80       (5.3s)
5   5_orders_by_status                  easy         1.00         1.00       1.00       1.00       (11.5s)
6   6_top_store_delivered               easy         1.00         1.00       1.00       1.00       (17.0s)
7   7_store_performance_scorecard       medium       1.00         1.00       1.00       0.70       (37.6s)
8   8_de

## Optimizing the Agent

DSPy optimizers automatically improve your AI program's performance by tuning prompts and model weights — so instead of manually tweaking prompt wording through trial and error, you let an algorithm systematically find what works best for your specific task and metric.

They take three inputs: 
- Your DSPy program (simple or complex)
- A scoring metric that defines what "good" looks like
- Set of training examples (as few as 5–10, labels optional)

From there, the optimizer iterates and refines your program to maximize that metric — achieving results that are often better than hand-crafted prompts and far more reproducible.

In [12]:
def agent_metric(example, pred, trace=None) -> float:
    """Score agent output using the combined LLM judge."""
    agent_sql = get_last_analytical_sql(pred.trajectory)
    if not agent_sql:
        return 0.0
    result = answer_quality(
        agent_sql,
        example.reference_sql,
        example.expected_answer,
        example.question,
        db_conn,
    )
    return result["score"]

In [13]:
# Build training set from eval dataset (exclude impossible questions)
trainset = [
    dspy.Example(
        question=ex["question"],
        db_context=DB_CONTEXT,
        expected_answer=ex["expected_answer"],
        reference_sql=ex["reference_sql"],
    ).with_inputs("question", "db_context")
    for ex in DATASET
    if not ex["difficulty"] == "impossible"
]

print(f"Training set: {len(trainset)} examples")

Training set: 14 examples


### GEPA (Genetic-Pareto Reflective Prompt Evolution) Optimizer

- GEPA (Genetic-Pareto) is a reflective prompt optimizer that improves your AI program by actually understanding what went wrong — not just trying random variations. Traditional optimizers and RL methods rely on scalar reward signals (just a number saying "good" or "bad"), which means they need thousands of rollouts to stumble toward better prompts. 
- GEPA instead reads the full execution traces — reasoning steps, tool outputs, error messages — and uses an LLM to reflect on why something failed and propose targeted fixes. Think of it as the difference between a student who only sees their test score versus one who gets detailed feedback on each answer.

Why it matters: 
- GEPA outperforms GRPO (a leading RL method) by 6% on average and up to 20%, while using up to 35x fewer rollouts. It also beats MIPROv2 by over 10%. [arXiv](https://arxiv.org/abs/2507.19457) 
- It achieves this through evolutionary search combined with Pareto-aware selection — keeping the best-performing prompt variants across multiple objectives simultaneously, rather than optimizing for a single metric.

A key advantage is that GEPA supports domain-specific textual feedback. Your metric function can return not just a score but a natural language explanation of what needs improvement, which GEPA uses directly to guide its next round of prompt evolution. This makes it especially powerful for complex, multi-module programs where you want fine-grained control over how each component improves.

In [20]:
# GEPA requires a 5-argument metric: (gold, pred, trace, pred_name, pred_trace)
def gepa_metric(example, pred, trace=None, pred_name=None, pred_trace=None) -> float:
    """Score agent output using the combined LLM judge (GEPA-compatible signature)."""
    agent_sql = get_last_analytical_sql(pred.trajectory)
    if not agent_sql:
        return 0.0
    result = answer_quality(
        agent_sql,
        example.reference_sql,
        example.expected_answer,
        example.question,
        db_conn,
    )
    return result["score"]


# Optimize agent_v2 with GEPA
gepa_optimizer = dspy.GEPA(
    metric=gepa_metric,
    max_full_evals=3,
    reflection_lm=lm,  # LM used for reflective reasoning
    num_threads=4,  # parallelize metric evaluations
    use_merge=False,  # skip merge phase
    reflection_minibatch_size=2,  # fewer examples per reflection step
)

agent_v2_fresh = dspy.ReAct(AnalystSignature, tools=[execute_sql], max_iters=10)
gepa_agent = gepa_optimizer.compile(agent_v2_fresh, trainset=trainset)
print("GEPA optimization complete.")

2026/02/26 21:41:39 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 42 metric calls of the program. This amounts to 3.00 full evals on the train set.
2026/02/26 21:41:39 WARNING dspy.teleprompt.gepa.gepa: No valset provided; Using trainset as valset. This is useful as an inference-time scaling strategy where you want GEPA to find the best solutions for the provided tasks in the trainset, as it makes GEPA overfit prompts to the provided trainset. In order to ensure generalization and perform well on unseen tasks, please provide separate trainset and valset. Provide the smallest valset that is just large enough to match the downstream task distribution, while keeping trainset as large as possible.
2026/02/26 21:41:39 INFO dspy.teleprompt.gepa.gepa: Using 14 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just l

Average Metric: 1.50 / 2 (75.0%): 100%|██████████| 2/2 [02:22<00:00, 71.39s/it] 

2026/02/26 21:45:42 INFO dspy.evaluate.evaluate: Average Metric: 1.5 / 2 (75.0%)


2026/02/26 21:46:01 WARNING dspy.teleprompt.gepa.gepa_utils: The score returned by the metric with pred_name is different from the overall metric score. This can indicate 2 things: Either the metric is non-deterministic (e.g., LLM-as-judge, Semantic score, etc.) or the metric returned a score specific to pred_name that differs from the module level score. Currently, GEPA does not support predictor level scoring (support coming soon), and only requires a feedback text to be provided, which can be specific to the predictor or program level. GEPA will ignore the differing score returned, and instead use module level score. You can safely ignore this warning if using a semantic metric, however, if this mismatch is caused due to predictor scoring, please return module-level scores. To disable this warning, set warn_on_score_mismatch=False.
2026/02/26 21:46:08 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for react: You are a data analyst working with a DuckDB database.

You

Average Metric: 0.80 / 2 (40.0%): 100%|██████████| 2/2 [00:48<00:00, 24.08s/it]

2026/02/26 21:50:58 INFO dspy.evaluate.evaluate: Average Metric: 0.8 / 2 (40.0%)


2026/02/26 21:51:06 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Proposed new text for extract.predict: You are a data analyst working with a DuckDB database.

Given a user's question, write and execute SQL queries and provide a
clear, accurate answer grounded in the data.

Do NOT waste iterations on SHOW TABLES or DESCRIBE — go straight to analytical queries.

When providing your final answer, do not include an `answer` key or similar. Respond directly with the answer text itself. Any output that is not recognized as a tool call will be considered the final answer.
2026/02/26 21:51:50 INFO dspy.evaluate.evaluate: Average Metric: 1.9 / 2 (95.0%)
2026/02/26 21:51:50 INFO dspy.teleprompt.gepa.gepa: Iteration 2: New subsample score 1.9 is better than old score 0.8. Continue to full eval and add to candidate pool.
2026/02/26 21:53:33 INFO dspy.evaluate.evaluate: Average Metric: 9.7 / 14 (69.3%)
2026/02/26 21:53:33 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Valset score for new program: 0

GEPA optimization complete.


2026/02/26 21:56:01 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=100000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently None)  if the reason for truncation is repetition.


In [ ]:
# Save the optimized agent
gepa_agent.save("gepa_agent.json")

In [15]:
# DSPy LM via OpenRouter
lm = dspy.LM(
    "openrouter/google/gemini-2.5-flash-lite",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    max_tokens=100000,
    cache=False,
)
dspy.configure(lm=lm, adapter=dspy.ChatAdapter(), track_usage=True)


In [16]:
eval_results = run_eval(
    agent_v2, DATASET, db_conn, agent_name="agent_v2", db_context=DB_CONTEXT
)

Running eval on 16 examples with agent_v2

#   ID                                  Difficulty   Efficiency   SQLValid   Recovery   Answer Quality
----------------------------------------------------------------------------------------------------
1   1_product_count                     easy         1.00         1.00       1.00       1.00       (6.8s)
2   2_unique_categories                 easy         1.00         1.00       1.00       1.00       (4.0s)
3   3_store_count                       easy         1.00         1.00       1.00       0.90       (4.1s)
4   4_avg_age_by_gender                 easy         1.00         1.00       1.00       0.80       (5.6s)
5   5_orders_by_status                  easy         1.00         1.00       1.00       1.00       (5.5s)
6   6_top_store_delivered               easy         1.00         1.00       1.00       0.00       (7.8s)
7   7_store_performance_scorecard       medium       1.00         1.00       1.00       0.70       (37.6s)
8   8_dema

In [ ]:
# Load it back later
pregenerated_gepa_agent = dspy.ReAct(
    AnalystSignature, tools=[execute_sql], max_iters=10
)
pregenerated_gepa_agent.load("pregenerated_gepa_agent.json")

In [ ]:
eval_results = run_eval(
    pregenerated_gepa_agent,
    DATASET,
    db_conn,
    agent_name="pregenerated_gepa_agent",
    db_context=DB_CONTEXT,
)

Running eval on 16 examples with gepa_agent

#   ID                                  Difficulty   Efficiency   SQLValid   Recovery   Answer Quality
----------------------------------------------------------------------------------------------------
1   1_product_count                     easy         1.00         1.00       1.00       1.00       (11.0s)
2   2_unique_categories                 easy         1.00         1.00       1.00       1.00       (8.0s)
3   3_store_count                       easy         1.00         1.00       1.00       1.00       (3.6s)
4   4_avg_age_by_gender                 easy         1.00         1.00       1.00       0.80       (4.7s)
5   5_orders_by_status                  easy         1.00         1.00       1.00       1.00       (21.1s)
6   6_top_store_delivered               easy         1.00         1.00       1.00       1.00       (8.6s)
7   7_store_performance_scorecard       medium       1.00         1.00       1.00       0.70       (68.5s)
8   8_